In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument('-headless')

browser = webdriver.Firefox(options=firefox_options)

start_t = time.time()
url = 'https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=9'
browser.get(url)

dfs = pd.read_html(str(browser.page_source))
run_t = time.time - start_t
print(f"run time: {run_t:.2f} mins") 
# 18.1

In [ ]:
dfs[3]

In [3]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

# Set options for headless mode
firefox_options = webdriver.FirefoxOptions()
firefox_options.add_argument('-headless')

# Initialize Firefox WebDriver with headless option
driver = webdriver.Firefox(options=firefox_options)

# URL pattern
base_url = "https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue={}&Division={}&Page={}"

# Create an empty DataFrame
columns = ["snapshot_periods", "weight_classes", "rank", "name", "country", "points"]
data = []

weight_classes_range = range(1, 13)  # define weightclasses to obtain


for division in weight_classes_range:
    issue = 878 # needs any arbitrary issue to get full list of issues from the dropdown
    driver.get(base_url.format(issue, division, 1)) 
    try:
        snapshot_dropdown = Select(driver.find_element(By.NAME, "Issue"))
        snapshot_periods = [option.get_attribute("value") for option in snapshot_dropdown.options if option.get_attribute("value")]
        max_pages_per_snapshot = 25
        start_t = time.time()
        run_t = 0
        for issue in snapshot_periods:
            page = 1
            while page <= max_pages_per_snapshot: 
                url = base_url.format(issue, division, page)
                driver.get(url)
                print("Current URL:", url)
                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//table[@class='tblRank']//tbody/tr")))
                    rows = driver.find_elements(By.XPATH, "//tbody/tr[td[@class='tdRank']]")
                    rows_alt = driver.find_elements(By.XPATH, "//tbody/tr[td[@class='tdRankAlt']]")
                    combined_rows = rows + rows_alt
                    print("Number of rows:", len(combined_rows))
                    if len(combined_rows) == 0:
                        run_t = (time.time() - start_t)/60 - run_t
                        print(f"snapshot done: {run_t:.2f} mins")  
                        break
                    for row in rows:
                        rank = row.find_element(By.CLASS_NAME, "tdRank").text
                        name = row.find_element(By.CLASS_NAME, "sherLink").text
                        country = row.find_element(By.TAG_NAME, "img").get_attribute("src").split("/")[-1].split(".")[0]
                        points = row.find_element(By.CLASS_NAME, "tdBar").text
                        data.append([issue, division, rank, name, country, points])
                    for row in rows_alt:
                        rank = row.find_element(By.CLASS_NAME, "tdRankAlt").text
                        name = row.find_element(By.CLASS_NAME, "sherLink").text
                        country = row.find_element(By.TAG_NAME, "img").get_attribute("src").split("/")[-1].split(".")[0]
                        points = row.find_element(By.CLASS_NAME, "tdBar").text
                        data.append([issue, division, rank, name, country, points])
                except NoSuchElementException:
                    print("No table rows found on this page.")
                page += 1   
    except NoSuchElementException:
        print("Element with name 'Issue' not found on the page")

df = pd.DataFrame(data, columns=columns)


driver.quit()

print(df)

Current URL: https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=1
Number of rows: 25
Current URL: https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=2
Number of rows: 25
Current URL: https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=3
Number of rows: 25
Current URL: https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=4
Number of rows: 25
Current URL: https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=5
Number of rows: 25
Current URL: https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=6
Number of rows: 25
Current URL: https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/?Issue=878&Division=1&Page=7
Number of rows: 25
Current URL: https://www.fightmatrix.com/historical-mma-rankings/ranking-snapshots/

KeyboardInterrupt: 

In [4]:
driver.quit()